# **Creating a Neural Network for handwritten Digit Recognition with PyTorch**

Author: Javier Eguíluz Romero

## Libraries and Settings

In [1]:
import torch
import numpy as np
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch import nn
from torch import optim
from tqdm import tqdm

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Function definitions

### Definition of the Neural Network

In [3]:
class MNIST_NN(nn.Module):
  def __init__(self, size_input, size_output):
    super(MNIST_NN, self).__init__()
    self.fc1 = nn.Linear(size_input, 400)
    self.rel1 = nn.ReLU()
    self.fc2 = nn.Linear(400, 50)
    self.rel2 = nn.ReLU()
    self.fc3 = nn.Linear(50,size_output)

  def forward(self, x):
    x = self.fc1(x)
    x = self.rel1(x)
    x = self.fc2(x)
    x = self.rel2(x)
    x = self.fc3(x)
    return x

### Helper Functions

In [4]:
def LoadData(batch_size, train_bool=True):
  data = DataLoader(
      datasets.MNIST(root='dataset/', train=train_bool, transform=transforms.ToTensor(), download=True),
      batch_size=batch_size, 
      shuffle=True)    
  return data

In [5]:

def train(data_train, model, criterion, optimizer, epochs):
  for epoch in tqdm(range(epochs)):
    for data, targets in data_train:
      data = data.to(device)
      targets = targets.to(device)
      data = data.reshape(data.shape[0], -1)

      # forward
      scores = model(data)
      loss = criterion(scores, targets)

      #backward
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

In [6]:
def performance_metrics(model, dataset):
  true_positives = [0]*10
  false_negatives = [0]*10
  false_positives = [0]*10

  model.eval()
  with torch.no_grad():
    for data, targets in dataset:
      data = data.to(device)
      targets = targets.to(device)
      data = data.reshape(data.shape[0], -1)

      scores = model(data)
      _, predicted = scores.max(1)

      for i in range(targets.size()[0]):
        actual_value = targets[i]
        predicted_value = predicted[i]

        if predicted_value == actual_value:
          true_positives[actual_value] += 1
        else:
          false_negatives[actual_value] += 1
          false_positives[predicted_value] += 1

  model.train()

  #precision
  precision = np.divide( true_positives, np.add(true_positives,false_positives) )
  #recall
  recall = np.divide( true_positives, np.add(true_positives,false_negatives) )
  #F1 Score
  F1_score = 2*np.divide( np.multiply(precision,recall), np.add(precision,recall) )

  return precision, recall, F1_score

## Training and evaluation of the Neural Network

### Loading the data from the MNIST Dataset

In [7]:
train_data = LoadData(batch_size=64, train_bool=True)
test_data = LoadData(batch_size=64, train_bool=False)

### Training of the model

In [8]:
model = MNIST_NN(size_input=784, size_output=10).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

train(train_data, model, criterion, optimizer, epochs=5)

100%|██████████| 5/5 [01:01<00:00, 12.28s/it]


### Testing of the model

In [9]:
precision, recall, F1_score = performance_metrics(model,test_data)
print(f'The performance of the model on the test data gives these averaged values: Precision: {np.mean(precision)*100:.2f}%, Recall: {np.mean(recall)*100:.2f}%, F1 score: {np.mean(F1_score):.4f}.')

The performance of the model on the test data gives these averaged values: Precision: 97.88%, Recall: 97.84%, F1 score: 0.9786.
